In [2]:
import gdown
!gdown 1nXAGmK_671MfcaemxHIERj9jaj9QF47g
!gdown 1Kz9xn8Ad6hVuIb0Y_oMlKvUxo8YLuBn4

Downloading...
From: https://drive.google.com/uc?id=1nXAGmK_671MfcaemxHIERj9jaj9QF47g
To: /content/annotations.zip
100% 8.84k/8.84k [00:00<00:00, 26.1MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1Kz9xn8Ad6hVuIb0Y_oMlKvUxo8YLuBn4
From (redirected): https://drive.google.com/uc?id=1Kz9xn8Ad6hVuIb0Y_oMlKvUxo8YLuBn4&confirm=t&uuid=10bf8544-1cff-4b61-a73b-dfb49125203e
To: /content/00.alumnes_practico.zip
100% 104M/104M [00:01<00:00, 88.3MB/s] 


In [4]:
!unzip /content/annotations.zip
!unzip /content/00.alumnes_practico.zip

Archive:  /content/annotations.zip
   creating: annotations/
  inflating: annotations/IMG_20240630_174459285.txt  
  inflating: annotations/IMG_20240630_173929084.txt  
  inflating: annotations/IMG_20240630_173836085.txt  
  inflating: annotations/IMG_20240630_173737533.txt  
  inflating: annotations/IMG_20240630_174115784.txt  
  inflating: annotations/IMG_20240630_173859456.txt  
  inflating: annotations/IMG_20240630_174155383_HDR.txt  
  inflating: annotations/IMG_20240630_173828513_HDR.txt  
  inflating: annotations/IMG_20240630_174333447.txt  
  inflating: annotations/IMG_20240630_174142929_HDR.txt  
  inflating: annotations/IMG_20240630_173822184_HDR.txt  
  inflating: annotations/IMG_20240630_174208092.txt  
  inflating: annotations/IMG_20240630_174349395.txt  
  inflating: annotations/IMG_20240630_174514649_HDR.txt  
  inflating: annotations/IMG_20240630_174316833.txt  
  inflating: annotations/IMG_20240630_173918579.txt  
  inflating: annotations/IMG_20240630_174406256.txt  
 

In [11]:
import os
import numpy as np
from collections import defaultdict

def parse_annotation_file(file_path, with_confidence=False):
    boxes = []
    with open(file_path, 'r') as file:
        for line in file:
            values = list(map(float, line.strip().split()))
            if with_confidence:
                class_id, x_center, y_center, width, height, confidence = values
                boxes.append([class_id, x_center, y_center, width, height, confidence])
            else:
                class_id, x_center, y_center, width, height = values
                boxes.append([class_id, x_center, y_center, width, height])
    return boxes

def parse_annotation_file(file_path, with_confidence=False):
    boxes = []
    with open(file_path, 'r') as file:
        for line in file:
            values = list(map(float, line.strip().split()))
            if with_confidence:
                class_id, x_center, y_center, width, height, confidence = values
                boxes.append((class_id, x_center, y_center, width, height, confidence))
            else:
                class_id, x_center, y_center, width, height = values
                boxes.append((class_id, x_center, y_center, width, height))
    return boxes

def compute_iou(box1, box2):
    x1_min = box1[1] - box1[3] / 2
    x1_max = box1[1] + box1[3] / 2
    y1_min = box1[2] - box1[4] / 2
    y1_max = box1[2] + box1[4] / 2

    x2_min = box2[1] - box2[3] / 2
    x2_max = box2[1] + box2[3] / 2
    y2_min = box2[2] - box2[4] / 2
    y2_max = box2[2] + box2[4] / 2

    inter_area = max(0, min(x1_max, x2_max) - max(x1_min, x2_min)) * \
                 max(0, min(y1_max, y2_max) - max(y1_min, y2_min))

    box1_area = box1[3] * box1[4]
    box2_area = box2[3] * box2[4]

    union_area = box1_area + box2_area - inter_area

    iou = inter_area / union_area
    return iou

def calculate_map(ground_truth_folder, prediction_folder, iou_thresholds=[0.5]):
    iou_results = defaultdict(list)
    for filename in os.listdir(ground_truth_folder):
        gt_file = os.path.join(ground_truth_folder, filename)
        pred_file = os.path.join(prediction_folder, filename)

        ground_truth_boxes = parse_annotation_file(gt_file)
        predicted_boxes = parse_annotation_file(pred_file, with_confidence=True)

        predicted_boxes = sorted(predicted_boxes, key=lambda x: -x[5])  # Sort by confidence score

        for iou_threshold in iou_thresholds:
            tp, fp = 0, 0
            matched_gt = set()

            for pred_box in predicted_boxes:
                pred_class_id = pred_box[0]
                best_iou = 0
                best_gt_box = None
                for gt_box in ground_truth_boxes:
                    gt_class_id = gt_box[0]
                    if gt_class_id == pred_class_id:
                        iou = compute_iou(pred_box, gt_box)
                        if iou > best_iou:
                            best_iou = iou
                            best_gt_box = gt_box

                # Convert ground truth box to tuple for set operations
                best_gt_box = tuple(best_gt_box) if best_gt_box else None

                if best_iou >= iou_threshold and best_gt_box not in matched_gt:
                    tp += 1
                    matched_gt.add(best_gt_box)
                else:
                    fp += 1

            fn = len(ground_truth_boxes) - len(matched_gt)
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            iou_results[iou_threshold].append((precision, recall))

    map_results = {}
    for iou_threshold, results in iou_results.items():
        precisions, recalls = zip(*results)
        average_precision = np.mean(precisions)
        map_results[iou_threshold] = average_precision

    return map_results

# Usage
ground_truth_folder = '/content/00.alumnes_practico/data/eval/labels/val'
prediction_folder = '/content/annotations'
iou_thresholds = [0.5, 0.75, 0.95]

map_results = calculate_map(ground_truth_folder, prediction_folder, iou_thresholds)

for iou, map_value in map_results.items():
    print(f"mAP at IoU {iou}: {map_value:.4f}")


mAP at IoU 0.5: 0.8333
mAP at IoU 0.75: 0.7240
mAP at IoU 0.95: 0.1250


In [15]:
import numpy as np
from collections import defaultdict

def compute_iou(pred_box, gt_box):
    # Unpack the predicted and ground truth boxes
    pred_class_id, pred_x_center, pred_y_center, pred_width, pred_height, _ = pred_box
    gt_class_id, gt_x_center, gt_y_center, gt_width, gt_height = gt_box

    # Convert to corner coordinates (x1, y1, x2, y2)
    pred_x1 = pred_x_center - (pred_width / 2)
    pred_y1 = pred_y_center - (pred_height / 2)
    pred_x2 = pred_x_center + (pred_width / 2)
    pred_y2 = pred_y_center + (pred_height / 2)

    gt_x1 = gt_x_center - (gt_width / 2)
    gt_y1 = gt_y_center - (gt_height / 2)
    gt_x2 = gt_x_center + (gt_width / 2)
    gt_y2 = gt_y_center + (gt_height / 2)

    # Calculate intersection coordinates
    inter_x1 = max(pred_x1, gt_x1)
    inter_y1 = max(pred_y1, gt_y1)
    inter_x2 = min(pred_x2, gt_x2)
    inter_y2 = min(pred_y2, gt_y2)

    # Calculate intersection area
    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)

    # Calculate union area
    pred_area = (pred_x2 - pred_x1) * (pred_y2 - pred_y1)
    gt_area = (gt_x2 - gt_x1) * (gt_y2 - gt_y1)
    union_area = pred_area + gt_area - inter_area

    # Calculate IoU
    iou = inter_area / union_area if union_area > 0 else 0
    return iou

def parse_annotation_file(file_path, with_confidence=False):
    boxes = []
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if with_confidence:
                class_id, x_center, y_center, width, height, confidence = map(float, parts)
                boxes.append([class_id, x_center, y_center, width, height, confidence])
            else:
                class_id, x_center, y_center, width, height = map(float, parts)
                boxes.append([class_id, x_center, y_center, width, height])
    return boxes

def calculate_metrics(ground_truth_folder, prediction_folder, iou_thresholds=[0.5]):
    iou_results = defaultdict(list)
    for filename in os.listdir(ground_truth_folder):
        gt_file = os.path.join(ground_truth_folder, filename)
        pred_file = os.path.join(prediction_folder, filename)

        ground_truth_boxes = parse_annotation_file(gt_file)
        predicted_boxes = parse_annotation_file(pred_file, with_confidence=True)

        predicted_boxes = sorted(predicted_boxes, key=lambda x: -x[5])  # Sort by confidence score

        for iou_threshold in iou_thresholds:
            tp, fp, fn = 0, 0, 0
            matched_gt = set()

            for pred_box in predicted_boxes:
                pred_class_id = pred_box[0]
                best_iou = 0
                best_gt_box = None
                for gt_box in ground_truth_boxes:
                    gt_class_id = gt_box[0]
                    if gt_class_id == pred_class_id:
                        iou = compute_iou(pred_box, gt_box)
                        if iou > best_iou:
                            best_iou = iou
                            best_gt_box = gt_box

                # Convert ground truth box to tuple for set operations
                best_gt_box = tuple(best_gt_box) if best_gt_box else None

                if best_iou >= iou_threshold and best_gt_box not in matched_gt:
                    tp += 1
                    matched_gt.add(best_gt_box)
                else:
                    fp += 1

            fn = len(ground_truth_boxes) - len(matched_gt)
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

            iou_results[iou_threshold].append((precision, recall, f1_score))

    metrics_results = {}
    for iou_threshold, results in iou_results.items():
        precisions, recalls, f1_scores = zip(*results)
        average_precision = np.mean(precisions)
        average_recall = np.mean(recalls)
        average_f1_score = np.mean(f1_scores)
        metrics_results[iou_threshold] = {
            'average_precision': average_precision,
            'average_recall': average_recall,
            'average_f1_score': average_f1_score
        }

    return metrics_results

metrics_results = calculate_metrics(ground_truth_folder, prediction_folder, iou_thresholds=[0.5, 0.75])
for iou, metrics in metrics_results.items():
    print(f"IoU Threshold: {iou}")
    print(f"Average Precision: {metrics['average_precision']:.4f}")
    print(f"Average Recall: {metrics['average_recall']:.4f}")
    print(f"Average F1 Score: {metrics['average_f1_score']:.4f}")


IoU Threshold: 0.5
Average Precision: 0.8333
Average Recall: 0.8698
Average F1 Score: 0.8448
IoU Threshold: 0.75
Average Precision: 0.7240
Average Recall: 0.7604
Average F1 Score: 0.7354


In [ ]:
map_results